### Test tools and functionsfor the ambient AI agent team experiment

In [6]:
from slack_tools import slack_communication

from datetime import datetime
import pytz

In [7]:
result = slack_communication(
    "Current time in Budapest and New York:\n"
    f"🇭🇺 Budapest: {datetime.now(pytz.timezone('Europe/Budapest')).strftime('%Y-%m-%d %H:%M')}\n"
    f"🗽 New York: {datetime.now(pytz.timezone('America/New_York')).strftime('%Y-%m-%d %H:%M')}"
)

In [8]:
# Example usage with a local image:
feedback = slack_communication(
    "How's your day going?", 
    require_feedback=True,
)

print(f"Received feedback: {feedback}")

Message sent, waiting for feedback (timeout: 60s)...
Received feedback: good


In [62]:
# Example usage with a local image:
result = slack_communication(
    "yo check it out", 
    image_path="../../output/social_media_experiment/AI rats overtaking New York City_img.png"
)

In [63]:
# Example with local image and waiting for feedback:
feedback = slack_communication(
    "What do you think of this local image?", 
    require_feedback=True,
    image_path="../../output/social_media_experiment/AI rats overtaking New York City_img.png"
)
print(f"Received feedback: {feedback}")

Message sent, waiting for feedback (timeout: 60s)...
Received feedback: yppppe


In [64]:
# Example usage with a local image:
result = slack_communication(
    "Here's an image from the internet", 
    image_url="https://miro.medium.com/v2/resize:fit:549/1*g-YZo7s0j46lDQfMmQ955A.png"
)
print(result)

Here's an image from the internet


In [65]:
# Example usage with a local image:
feedback = slack_communication(
    "what do you see here bro?", 
    require_feedback=True,
    image_url="https://miro.medium.com/v2/resize:fit:549/1*g-YZo7s0j46lDQfMmQ955A.png"
)

print(f"Received feedback: {feedback}")

Message sent, waiting for feedback (timeout: 60s)...
Received feedback: nice logo man!


#### Test image input to API

In [12]:
import os
import dotenv
dotenv.load_dotenv()

import base64
from openai import OpenAI
client = OpenAI()

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Path to your image
image_path = "stock_data/BUDML_FINAL.png"

# Getting the Base64 string
base64_image = encode_image(image_path)

In [13]:
for detail in ["low", "high"]:
    
    completion = client.chat.completions.create(
        model="o1-2024-12-17",
        messages=[
            {
                "role": "user",
                "content": [
                    { "type": "text", "text": "what's in this image?" },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": detail
                        },
                    },
                ],
            }
        ],
    )

    print(f"Detail: {detail}")
    print(completion.choices[0].message.content)
    print("\n\n")

Detail: low
It appears to be a line chart labeled “Stock Price Chart for $BUDML.” The x-axis is labeled “Time (seconds)” and the y-axis is labeled “Stock Price (USD).” The blue line shows a stock price starting around 1000 USD, dropping to about 900–950 in the middle of the time span, and then rising above 1200 USD by the end of the timeline.



Detail: high
It’s a line chart titled “Stock Price Chart for $BUDML,” showing how the stock’s price (on the y-axis, labeled in USD) changes over time in seconds (on the x-axis). The line starts near 1000 USD, dips to around 900 USD at about 70 seconds, then climbs sharply to around 1200 USD by the 150-second mark.





In [15]:
completion.__dict__

{'id': 'chatcmpl-BBoGP3KmJcuXNDh8Y3tz9VA42GCwT',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='It’s a line chart titled “Stock Price Chart for $BUDML,” showing how the stock’s price (on the y-axis, labeled in USD) changes over time in seconds (on the x-axis). The line starts near 1000 USD, dips to around 900 USD at about 70 seconds, then climbs sharply to around 1200 USD by the 150-second mark.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))],
 'created': 1742154521,
 'model': 'o1-2024-12-17',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_60442b0df0',
 'usage': CompletionUsage(completion_tokens=347, prompt_tokens=777, total_tokens=1124, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=256, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0